In [360]:
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime
devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
time_now = datetime.datetime.now()
time_now_formatted = time_now.strftime("%Y-%m-%d")

In [361]:
pd.read_sql_query(
    f"Select DISTINCT * From results_hard",con=devengine,)

,Winner,Winner_Odds,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,fav_percent,dog_percent,Sex
0,Joris De Loore,1.27,1.27,3.48,Joris De Loore,Joris De Loore,82%,11,Denis Yevseyev,42%,12,0.818182,0.416667,Mens
1,Maxime Cressy,2.43,1.56,2.43,Emil Ruusuvuori,Emil Ruusuvuori,48%,48,Maxime Cressy,47%,32,0.479167,0.468750,Mens
2,Cem Ilkel,1.63,1.63,2.2,Cem Ilkel,Cem Ilkel,52%,21,Laurent Lokoli,80%,10,0.523810,0.800000,Mens
3,Ricardas Berankis,1.48,1.48,2.53,Ricardas Berankis,Ricardas Berankis,64%,11,Altug Celikbilek,53%,17,0.636364,0.529412,Mens
4,Arthur Fils,3.74,1.28,3.74,Roberto Bautista Agut,Roberto Bautista Agut,80%,10,Arthur Fils,45%,11,0.800000,0.454545,Mens
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3490,Corentin Moutet,1.12,1.12,5.55,Corentin Moutet,Corentin Moutet,83%,12,Michael Geerts,40%,10,0.833333,0.400000,Mens
3491,Yoshihito Nishioka,1.62,1.62,2.3,Yoshihito Nishioka,Yoshihito Nishioka,44%,18,Taro Daniel,29%,17,0.444444,0.294118,Mens
3492,Mackenzie Mcdonald,1.08,1.08,7.61,Mackenzie Mcdonald,Mackenzie Mcdonald,69%,13,Shintaro Mochizuki,27%,11,0.692308,0.272727,Mens
3493,Jenson Brooksby,1.53,1.53,2.49,Jenson Brooksby,Jenson Brooksby,81%,16,Soon Woo Kwon,37%,27,0.812500,0.370370,Mens


In [362]:
data=pd.read_sql_query(
    f"Select DISTINCT Winner,Dog,Dog_Record,Winner_Odds,Fav_Record,fav_percent,dog_percent,Fav_Games,Dog_Games,Fav_Odds,Fav,sex,Dog_Odds From results_hard \
    where sex like 'Mens' --and Date > '2023-01-01'",
    con=devengine,)
data['Fav_Odds']=data['Fav_Odds'].astype(float)
data['Dog_Odds']=data['Dog_Odds'].astype(float)
data['Winx']=data.apply(lambda x: 'Dog' if x['Winner']==x['Dog'] else 'Fav', axis=1)
#print(len(data[(data['Winner']==data['Fav'])&(data['Fav_Odds']>1.9)])/len(data[data['Fav_Odds']>1.9]))
#data[data['Fav_Odds']>1.8].to_csv('test.csv')
#data=data[data['Fav_Odds']>1.19]
#data=data[data['fav_percent']>0.3]
#data=data[data['dog_percent']>0.1]

In [363]:
mensserving=pd.read_csv('mensserving.csv')
combine = pd.merge(data, mensserving, how="left", left_on="Fav", right_on="Name")
combine2 = pd.merge(combine, mensserving, how="left", left_on="Dog", right_on="Name")
combine2[["Service Games Won_x", "Service Games Won_y"]] = combine2[
        ["Service Games Won_x", "Service Games Won_y"]
    ].astype(float)
combine2.rename(
        columns={
            "Service Games Won_x": "Fav_Serve%",
            "Service Games Won_y": "Dog_Serve%",
            "Return Games Won_x": "Fav_Return%",
            "Return Games Won_y": "Dog_Return%",
        },
        inplace=True,
    )
mens = combine2[
    [   "Winner","Dog","Winner_Odds","fav_percent","dog_percent","Fav_Games","Dog_Games","Fav_Odds",
        "Fav",
        # "Player_1_Odds",
        "Fav_Serve%",
        "Dog_Return%",
       
        # "Player_2_Odds",
        "Dog_Serve%",
        "Fav_Return%","Dog_Odds"
    ]]

In [364]:
import numpy as np

print("><")
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
        k=x+0.1
        l=y-0.1
   
        mensfilter=mens[(mens['dog_percent']>x)&(mens['dog_percent']<k)&(mens['fav_percent']<y)&(mens['fav_percent']>l)]
        mensfiltertxt=f"print(final_hard['Dog'][(final_hard['dog_percent']>{x.round(1)})&(final_hard['dog_percent']<{k.round(1)})&(final_hard['fav_percent']<{y.round(1)})&(final_hard['fav_percent']>{l.round(1)})])"
        
        if len(mensfilter)>1 and len(mensfilter[mensfilter['Winner']==mensfilter['Dog']])/len(mensfilter) > 0.5:

            print(mensfiltertxt)
            print(len(mensfilter[mensfilter['Winner']==mensfilter['Dog']])/len(mensfilter))
            print(len(mensfilter))
print("<<")            
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
        k=x+0.1
        l=y-0.1
   
        mensfilter=mens[(mens['dog_percent']<x)&(mens['dog_percent']>k)&(mens['fav_percent']<y)&(mens['fav_percent']>l)]
        mensfiltertxt=f"print(final_hard['Dog'][(final_hard['dog_percent']<{x.round(1)})&(final_hard['dog_percent']>{k.round(1)})&(final_hard['fav_percent']<{y.round(1)})&(final_hard['fav_percent']>{l.round(1)})])"

        
        if len(mensfilter)>1 and len(mensfilter[mensfilter['Winner']==mensfilter['Dog']])/len(mensfilter) > 0.5:
            
            print(mensfiltertxt)
            print(len(mensfilter[mensfilter['Winner']==mensfilter['Dog']])/len(mensfilter))
            print(len(mensfilter))      
print(">>")
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
        
   
        mensfilter=mens[(mens['dog_percent']>x)&(mens['fav_percent']>y)]
        if len(mensfilter)>1 and len(mensfilter[mensfilter['Winner']==mensfilter['Dog']])/len(mensfilter) > 0.5:
            print(x,y)
            print(len(mensfilter[mensfilter['Winner']==mensfilter['Dog']])/len(mensfilter))
            print(len(mensfilter))       
print("<>")
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
        
   
        mensfilter=mens[(mens['dog_percent']<x)&(mens['fav_percent']>y)]
        if len(mensfilter)>1 and len(mensfilter[mensfilter['Winner']==mensfilter['Dog']])/len(mensfilter) > 0.5:
            print(x,y)
            print(len(mensfilter[mensfilter['Winner']==mensfilter['Dog']])/len(mensfilter))
            print(len(mensfilter))    

><
print(final_hard['Dog'][(final_hard['dog_percent']>0.4)&(final_hard['dog_percent']<0.5)&(final_hard['fav_percent']<0.3)&(final_hard['fav_percent']>0.2)])
0.5714285714285714
7
print(final_hard['Dog'][(final_hard['dog_percent']>0.6)&(final_hard['dog_percent']<0.7)&(final_hard['fav_percent']<0.4)&(final_hard['fav_percent']>0.3)])
0.7142857142857143
7
print(final_hard['Dog'][(final_hard['dog_percent']>0.7)&(final_hard['dog_percent']<0.8)&(final_hard['fav_percent']<0.4)&(final_hard['fav_percent']>0.3)])
1.0
3
<<
>>
0.6000000000000001 0.9
0.7142857142857143
7
0.7000000000000001 0.9
1.0
2
<>


In [365]:
import numpy as np

print("><")
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
   
        mensfilter=mens[(mens['dog_percent']>x)&(mens['fav_percent']<y)&(mens['Fav_Odds']>1.19)&(mens['Fav_Odds']<1.3)]
        
        if len(mensfilter)>10 and len(mensfilter[mensfilter['Winner']!=mensfilter['Dog']])/len(mensfilter) > 0.9:

            print(x,y)
            print(len(mensfilter[mensfilter['Winner']!=mensfilter['Dog']])/len(mensfilter))
            print(len(mensfilter))
print("<<")            
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
   
        mensfilter=mens[(mens['dog_percent']<x)&(mens['fav_percent']<y)&(mens['Fav_Odds']>1.19)&(mens['Fav_Odds']<1.3)]
        
        if len(mensfilter)>10 and len(mensfilter[mensfilter['Winner']!=mensfilter['Dog']])/len(mensfilter) > 0.9:
            
            print(x,y)
            print(len(mensfilter[mensfilter['Winner']!=mensfilter['Dog']])/len(mensfilter))
            print(len(mensfilter))      
print(">>")
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
        
   
        mensfilter=mens[(mens['dog_percent']>x)&(mens['fav_percent']>y)&(mens['Fav_Odds']>1.19)&(mens['Fav_Odds']<1.3)]
        if len(mensfilter)>10 and len(mensfilter[mensfilter['Winner']!=mensfilter['Dog']])/len(mensfilter) > 0.9:
            print(x,y)
            print(len(mensfilter[mensfilter['Winner']!=mensfilter['Dog']])/len(mensfilter))
            print(len(mensfilter))       
print("<>")
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
        
   
        mensfilter=mens[(mens['dog_percent']<x)&(mens['fav_percent']>y)&(mens['Fav_Odds']>1.19)&(mens['Fav_Odds']<1.3)]
        if len(mensfilter)>10 and len(mensfilter[mensfilter['Winner']!=mensfilter['Dog']])/len(mensfilter) > 0.9:
            print(x,y)
            print(len(mensfilter[mensfilter['Winner']!=mensfilter['Dog']])/len(mensfilter))
            print(len(mensfilter))    

><
0.0 0.4
0.9230769230769231
13
0.1 0.4
0.9230769230769231
13
0.1 0.5
0.9166666666666666
24
0.2 0.4
0.9090909090909091
11
0.2 0.5
0.9047619047619048
21
<<
0.6000000000000001 0.4
0.9230769230769231
13
0.7000000000000001 0.4
0.9230769230769231
13
0.8 0.4
0.9230769230769231
13
0.9 0.4
0.9230769230769231
13
>>
0.5 0.8
0.9473684210526315
19
<>


In [366]:
mensfilter=mens[(mens['Fav_Odds']>1.8)]
print(len(mensfilter[mensfilter['Winner']!=mensfilter['Dog']])/len(mensfilter))
print(len(mensfilter))

0.5294117647058824
153


In [367]:
test=mens[(mens['Fav_Serve%']<mens['Dog_Serve%'])&(mens['Dog_Return%']>20)&(mens['Fav_Serve%']>0)&(mens['Dog_Odds']<2.5)]
len(test[test['Winner']==test['Dog']])/len(test)
#len(test)
#test

0.671875

In [368]:
data=pd.read_sql_query(
    f"Select DISTINCT Winner,Dog,Dog_Record,Winner_Odds,Fav_Record,fav_percent,dog_percent,Fav_Games,Dog_Games,Fav_Odds,Fav,sex,Dog_Odds From results_hard where sex like 'Womens' --where Date > '2023-01-01' and Date not like '{time_now_formatted}'",
    con=devengine,)
data['Fav_Odds']=data['Fav_Odds'].astype(float)
data['Dog_Odds']=data['Dog_Odds'].astype(float)
data['Winx']=data.apply(lambda x: 'Dog' if x['Winner']==x['Dog'] else 'Fav', axis=1)

In [369]:
womensserving=pd.read_csv('womensserving.csv')
combine = pd.merge(data, womensserving, how="left", left_on="Fav", right_on="Name")
combine2 = pd.merge(combine, womensserving, how="left", left_on="Dog", right_on="Name")
combine2[["Service Games Won_x", "Service Games Won_y"]] = combine2[
        ["Service Games Won_x", "Service Games Won_y"]
    ].astype(float)
combine2.rename(
        columns={
            "Service Games Won_x": "Fav_Serve%",
            "Service Games Won_y": "Dog_Serve%",
            "Return Games Won_x": "Fav_Return%",
            "Return Games Won_y": "Dog_Return%",
        },
        inplace=True,
    )
womens = combine2[
    [   "Winner","Dog","Winner_Odds","fav_percent","dog_percent","Fav_Games","Dog_Games","Fav_Odds",
        "Fav",
        # "Player_1_Odds",
        "Fav_Serve%",
        "Dog_Return%",
       
        # "Player_2_Odds",
        "Dog_Serve%",
        "Fav_Return%","Dog_Odds"
    ]]

In [370]:
womensfilter=womens[(womens['dog_percent']>0)&(womens['fav_percent']>0.5)&(womens['Fav_Serve%']>70)&(womens['Dog_Serve%']>0.0)&(womens['Dog_Serve%']<70.0)&(womens['Dog_Return%']>0)&(womens['Fav_Odds']>1.8)]
len(womensfilter[womensfilter['Winner']!=womensfilter['Dog']])/len(womensfilter)

0.25

In [371]:
len(womensfilter)

4

In [372]:
data3=womens[(womens['Fav_Odds']>1.19)&(womens['Fav_Odds']<1.3)&(womens['dog_percent']<0.3)&(womens['fav_percent']>0.6)&(womens['Dog_Games']>8)]
len(data3[data3['Winner']==data3['Fav']])/len(data3)

0.9285714285714286

In [373]:
len(data3)

14

In [374]:
data3

,Winner,Dog,Winner_Odds,fav_percent,dog_percent,Fav_Games,Dog_Games,Fav_Odds,Fav,Fav_Serve%,Dog_Return%,Dog_Serve%,Fav_Return%,Dog_Odds
18,Maria Sakkari,Nuria Parrizas Diaz,1.21,0.645161,0.272727,31,11,1.21,Maria Sakkari,74.8,35.2,60.1,33.0,4.37
35,Liudmila Samsonova,Claire Liu,1.29,0.613636,0.285714,44,21,1.29,Liudmila Samsonova,81.0,37.2,62.6,31.1,3.55
64,Mirjam Bjorklund,Mirjam Bjorklund,3.78,0.636364,0.200000,11,10,1.25,Aliaksandra Sasnovich,67.2,31.7,66.7,37.9,3.78
73,Caroline Garcia,Alison Van Uytvanck,1.27,0.730769,0.166667,26,12,1.27,Caroline Garcia,80.3,32.2,71.5,27.1,3.78
117,Xin Yu Wang,Joanne Zuger,1.24,0.769231,0.222222,13,9,1.24,Xin Yu Wang,NaN,30.0,43.3,NaN,4.07
206,Cori Gauff,Emma Raducanu,1.29,0.821429,0.181818,28,11,1.29,Cori Gauff,NaN,38.9,68.5,NaN,3.66
234,Petra Kvitova,Alison Van Uytvanck,1.24,0.692308,0.200000,26,10,1.24,Petra Kvitova,74.3,32.2,71.5,29.9,4.11
288,Clara Burel,Carol Zhao,1.21,0.636364,0.100000,11,10,1.21,Clara Burel,56.2,26.3,68.4,35.2,4.22
412,Kaia Kanepi,Anna Bondar,1.28,0.650000,0.266667,20,15,1.28,Kaia Kanepi,71.7,30.3,70.3,31.8,3.51
436,Cori Gauff,Sofia Kenin,1.21,0.818182,0.166667,11,12,1.21,Cori Gauff,NaN,23.3,63.5,NaN,4.42


In [375]:
import numpy as np

print("><")
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
   
        womensfilter=womens[(womens['dog_percent']>x)&(womens['fav_percent']<y)&(womens['Fav_Odds']>1.19)&(womens['Fav_Odds']<1.3)]
        
        if len(womensfilter)>10 and len(womensfilter[womensfilter['Winner']!=womensfilter['Dog']])/len(womensfilter) > 0.9:

            print(x,y)
            print(len(womensfilter[womensfilter['Winner']!=womensfilter['Dog']])/len(womensfilter))
            print(len(womensfilter))
print("<<")            
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
   
        womensfilter=womens[(womens['dog_percent']<x)&(womens['fav_percent']<y)&(womens['Fav_Odds']>1.19)&(womens['Fav_Odds']<1.3)]
        
        if len(womensfilter)>10 and len(womensfilter[womensfilter['Winner']!=womensfilter['Dog']])/len(womensfilter) > 0.9:
            
            print(x,y)
            print(len(womensfilter[womensfilter['Winner']!=womensfilter['Dog']])/len(womensfilter))
            print(len(womensfilter))      
print(">>")
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
        
   
        womensfilter=womens[(womens['dog_percent']>x)&(womens['fav_percent']>y)&(womens['Fav_Odds']>1.19)&(womens['Fav_Odds']<1.3)]
        if len(womensfilter)>10 and len(womensfilter[womensfilter['Winner']!=womensfilter['Dog']])/len(womensfilter) > 0.9:
            print(x,y)
            print(len(womensfilter[womensfilter['Winner']!=womensfilter['Dog']])/len(womensfilter))
            print(len(womensfilter))       
print("<>")
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
        
   
        womensfilter=womens[(womens['dog_percent']<x)&(womens['fav_percent']>y)&(womens['Fav_Odds']>1.19)&(womens['Fav_Odds']<1.3)]
        if len(womensfilter)>10 and len(womensfilter[womensfilter['Winner']!=womensfilter['Dog']])/len(womensfilter) > 0.9:
            print(x,y)
            print(len(womensfilter[womensfilter['Winner']!=womensfilter['Dog']])/len(womensfilter))
            print(len(womensfilter))    

><
<<
0.2 0.9
0.9090909090909091
11
>>
<>
0.2 0.0
0.9166666666666666
12
0.2 0.1
0.9166666666666666
12
0.2 0.2
0.9090909090909091
11
0.2 0.30000000000000004
0.9090909090909091
11


In [376]:
import numpy as np

print("><")
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
   
        womensfilter=womens[(womens['dog_percent']>x)&(womens['fav_percent']<y)]
        
        if len(womensfilter)>5 and len(womensfilter[womensfilter['Winner']==womensfilter['Dog']])/len(womensfilter) > 0.5:

            print(x,y)
            print(len(womensfilter[womensfilter['Winner']==womensfilter['Dog']])/len(womensfilter))
            print(len(womensfilter))
print("<<")            
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
   
        womensfilter=womens[(womens['dog_percent']<x)&(womens['fav_percent']<y)]
        
        if len(womensfilter)>10 and len(womensfilter[womensfilter['Winner']==womensfilter['Dog']])/len(womensfilter) > 0.5:
            
            print(x,y)
            print(len(womensfilter[womensfilter['Winner']==womensfilter['Dog']])/len(womensfilter))
            print(len(womensfilter))      
print(">>")
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
        
   
        womensfilter=womens[(womens['dog_percent']>x)&(womens['fav_percent']>y)]
        if len(womensfilter)>10 and len(womensfilter[womensfilter['Winner']==womensfilter['Dog']])/len(womensfilter) > 0.5:
            print(x,y)
            print(len(womensfilter[womensfilter['Winner']==womensfilter['Dog']])/len(womensfilter))
            print(len(womensfilter))       
print("<>")
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
        
   
        womensfilter=womens[(womens['dog_percent']<x)&(womens['fav_percent']>y)]
        if len(womensfilter)>10 and len(womensfilter[womensfilter['Winner']==womensfilter['Dog']])/len(womensfilter) > 0.5:
            print(x,y)
            print(len(womensfilter[womensfilter['Winner']==womensfilter['Dog']])/len(womensfilter))
            print(len(womensfilter))    

><
0.1 0.4
0.5053763440860215
93
0.2 0.4
0.5365853658536586
82
0.30000000000000004 0.4
0.532258064516129
62
0.4 0.30000000000000004
0.55
20
0.4 0.4
0.5897435897435898
39
0.5 0.4
0.6190476190476191
21
0.8 0.7000000000000001
0.7142857142857143
7
0.8 0.8
0.7142857142857143
7
0.8 0.9
0.7142857142857143
7
<<
0.6000000000000001 0.4
0.5057471264367817
87
0.7000000000000001 0.4
0.5108695652173914
92
>>
0.7000000000000001 0.5
0.5333333333333333
15
<>


In [378]:
test=womens[(womens['Fav_Serve%']>womens['Dog_Serve%'])&(womens['Dog_Return%']>10)&(womens['Fav_Serve%']>0)&(womens['Dog_Odds']<2.5)&(womens['Dog_Serve%']>70)]
len(test[test['Winner']!=test['Fav']])/len(test)
#len(test)
#test

27